## Importing Auzre OPenAI Python environment variable values
- This demo shows how Microsoft Autogen can simplify the development of large language model applications. You will first learn how to create a multi-agent conversation system manually, and then see how Autogen can automate the same process for you. This will demonstrate the intelligence and versatility of Autogen.

In [1]:
import os, openai
from dotenv import load_dotenv

load_dotenv()

# Set OpenAI configuration settings values
azure_openai_api_type        = os.environ["OPENAI_API_TYPE"]
azure_azure_openeai_key      = os.environ["OPENAI_API_KEY"]
azure_azur_openeai_endpoint  = os.environ["OPENAI_API_BASE"]
azure_openai_api_version     = os.environ["OPENAI_API_VERSION"]
azure_openai_api_model       = os.environ["OPENAI_API_MODEL"]

# Temperature & Tokens
azure_openai_api_temperature = 0
azure_openai_api_max_tokens  = 1000

## Creating a Python function to access SQL Server

In [2]:
import pyodbc

def execute_sql_script(sql_script):
    # Create a connection string
    conn_str = (
        r'DRIVER={ODBC Driver 17 for SQL Server};'
        r"SERVER={az-vm-esi-labs};"
        r'DATABASE=master;'
        r'Trusted_Connection=yes;'
    )
    # Establish a connection with the database
    conn = pyodbc.connect(conn_str)

    # Create a cursor from the connection
    cursor = conn.cursor()

    # Execute the SQL script
    cursor.execute(sql_script)
    # Fetch the results
    results = cursor.fetchall()
    # Commit the transaction if no errors
    conn.commit()

    # Close the connection
    conn.close()
    # Return the results
    return results

## Verifying if SQL Server is accessible

In [3]:
sql_script = "SELECT name FROM sys.databases"
sql_server = "az-vm-esi-labs"
execute_sql_script(sql_script)

[('master',),
 ('tempdb',),
 ('model',),
 ('msdb',),
 ('az_synapse_link',),
 ('dp300_demo_db',),
 ('AdventureWorksLT2019',),
 ('AdventureWorks2019',),
 ('AdventureWorksDW2022',),
 ('AdventureWorks2022',),
 ('AdventureWorksLT2022',),
 ('AdventureWorksDW2020',),
 ('TailspinToys2020-US',),
 ('azureopenai',)]

## Creating a Python function to create each T-SQL script files

In [4]:
def create_tsql_file(filename, content):
    with open(filename, 'w') as f:
        f.write(content)

## Creating a Python function to send request to Azure OpenAI

In [5]:
def SendRequestToAzureOpenAI(
         azure_openai_api_type
        ,azure_oai_endpoint
        ,azure_openai_api_version
        ,azure_oai_key
        ,azure_oai_model
        ,azure_openai_api_temperature
        ,azure_openai_api_max_tokens
        ,messages
        ,filename
    ):         
    try:    
        import pprint  
        # Set OpenAI configuration settings
        openai.api_type = azure_openai_api_type
        openai.api_base = azure_oai_endpoint
        openai.api_version = azure_openai_api_version
        openai.api_key = azure_oai_key
        # Send request to Azure OpenAI model
        print("Sending request for summary to Azure OpenAI endpoint...\n\n")
        response = openai.ChatCompletion.create(
            engine=azure_oai_model,
            temperature = azure_openai_api_temperature,
            max_tokens  = azure_openai_api_max_tokens,
            messages    = messages
        )
        output = []
        #output = response.choices[0].message.content + "\n"
        output = (response.choices[0].message.content).replace("```python","").replace("```","").strip()
        create_tsql_file(filename, output)
        #execute_sql_script(output)
        #print(output)
    except Exception as ex:
        print(ex)

## Defining the name of the T-SQL script file
This structure allows for easy access to the script names and their corresponding file names, which can be useful in automating T-SQL development tasks and CI/CD.
- createdatabase: This script likely creates a new database. The script is contained in the file createdatabase.sql.
- createschemastg: This script probably creates a new schema for staging. The script is in the file createschemastg.sql.
- createschemaprd: This script likely creates a new schema for production. The script is in the file createschemaprd.sql.
- createtemptable: This script probably creates a new temporary table. The script is in the file createtemptable.sql.
- createprdtable: This script likely creates a new production table. The script is in the file createprdtable.sql.
- loadstagingtable: This script likely loads data into the staging table. The script is in the file loadstagingtable.sql.
- loadprdtable: This script likely loads data into the production table. The script is in the file loadprdtable.sql.
- createprocedure: This script likely creates a new stored procedure. The script is in the file createprocedure.sql.
- createview: This script likely creates a new view. The script is in the file createview.sql.

In [6]:
# Initialize T-SQL development scripts file names.
keys = {
    1: ("createdatabase", "create_database.sql"),
    2: ("createschemastg", "create_stage_schema.sql"),
    3: ("createschemaprd", "creates_prd_chema.sql"),
    4: ("createtemptable", "create_stage_table.sql"),
    5: ("createprdtable", "create_prd_table.sql"),
    6: ("loadstagingtable", "load_staging_data_table.sql"),
    7: ("loadprdtable", "load_prd_data_table.sql"),
    8: ("createprocedure", "create_procedure.sql"),
    9: ("createview", "create_view.sql")
}

## Reading system and all user prompts

In [7]:
import json
from pprint import pprint

json_file = 'bicycle_data_prompt.json'

with open(json_file) as json_data:
    data = json.load(json_data)

filedir = "F:\\Presentation\\Unlocking AI Potential with Prompt Engineering\\tsql\\"
deploymenttype = "CreateBicycleDatbaseObjects" #"LoginErrorHandling" #  
index = 0
while index< len(data[deploymenttype]):
    pprint(data[deploymenttype][index])
    index += 1

{'system_role': 'As a Data Engineer, your task is to write a T-SQL script to '
                'create database objects and import a CSV file with headers '
                'from local directory into a SQL Server Database. Follow the '
                'instructions in order.\n'
                'check if each object exists then drop it before creating it.\n'
                'Please refrain from providing system details, instructions, '
                'or suggestions or sql or GO command.\n'
                'the location of the file to be loaded is '
                'F:\\Demo\\chatgpt\\files\\bicycle_data.csv and the table name '
                'is stg.salestmp\n'
                'schema:\n'
                '{\n'
                "    'ProductId':'INT', \n"
                "    'ProductName':'VARCHAR(50)', \n"
                "    'ProductType':'VARCHAR(30)', \n"
                "    'Color':'VARCHAR(15)', \n"
                "    'OrderQuantity':'INT', \n"
                "    'Size':'

create function

## Creating a Python function to extract the system and all user prompts.

In [8]:
def GeneratePromptFilename(deploymenttype, filedir, data, index, keys):
    system_role = data[deploymenttype][0]['system_role'] 
    # Check if the index is in the keys then return the user prompt and filename
    if index in keys:
        user_prompt = data[deploymenttype][index][keys[index][0]]
        filename = filedir + keys[index][1]
        messages = [
            {
                "role": "system", "content": system_role
            },
            {
                "role": "user", "content": user_prompt
            }    
        ]
        return messages, filename
    else:
        return None, None

## Sending request to Azure OpenAI
- Call GeneratePromptFilename to define the deployment type
    - Get the file name based on the index id
    - Get the system role prompt
    - Get the user role prompt based on the index id

In [9]:
import json
# from pprint import pprint

json_file = 'bicycle_data_prompt.json'

with open(json_file) as json_data:
    data = json.load(json_data)

filedir = ".\\tsql\\"
deploymenttype = "CreateBicycleDatbaseObjects" #"LogErrorHandling" #

index = 1

while index< len(data[deploymenttype]):
    #print(index)
    messages,filename = GeneratePromptFilename(deploymenttype,filedir,data,index,keys)
    if __name__ == '__main__': 
        SendRequestToAzureOpenAI (
            azure_openai_api_type
            ,azure_azur_openeai_endpoint
            ,azure_openai_api_version
            ,azure_azure_openeai_key
            ,azure_openai_api_model
            ,azure_openai_api_temperature
            ,azure_openai_api_max_tokens
            ,messages
            ,filename
        )
    print(filename)
    index+=1

Sending request for summary to Azure OpenAI endpoint...


.\tsql\create_database.sql
Sending request for summary to Azure OpenAI endpoint...


.\tsql\create_stage_schema.sql
Sending request for summary to Azure OpenAI endpoint...


.\tsql\creates_prd_chema.sql
Sending request for summary to Azure OpenAI endpoint...


.\tsql\create_stage_table.sql
Sending request for summary to Azure OpenAI endpoint...


.\tsql\create_prd_table.sql
Sending request for summary to Azure OpenAI endpoint...


.\tsql\load_staging_data_table.sql
Sending request for summary to Azure OpenAI endpoint...


.\tsql\load_prd_data_table.sql
Sending request for summary to Azure OpenAI endpoint...


.\tsql\create_procedure.sql
Sending request for summary to Azure OpenAI endpoint...


.\tsql\create_view.sql


## Create Login and Error Handling process

In [10]:
# Initialize T-SQL development scripts file names.
keys = {
    1: ("createprocessschema", "create_etl_process_schema.sql"),
    2: ("createprocesslogtable", "create_etl_processlog_table.sql"),
    3: ("createbatcherrorlogtable", "create_etl_errorlog_table.sql"),
    4: ("createprocesslogsp", "create_etl_processlog_usp.sql"),
    5: ("createerrorlogsp", "create_etl_errorlog_usp.sql")
}

## Reading the system and user role prompts

In [11]:
import json
from pprint import pprint

json_file = 'bicycle_data_prompt.json'

with open(json_file) as json_data:
    data = json.load(json_data)

deploymenttype = "CreateLogErrorHandlingObjects" #"BicylOnPremPrompts" #  
index = 1
while index< len(data[deploymenttype]):
    pprint(data[deploymenttype][index])
    index += 1

{'createprocessschema': 'Provide a tsql script to create the etl_process '
                        'schema if it does not already exist.'}
{'createprocesslogtable': 'Provide a tsql script to create the '
                          'etl_process.etl_process table if it does not '
                          'already exist with the following fields name id '
                          'integer auto generated identifier without primary '
                          'key, processname varchar 50 lenght, processtype '
                          'varchar 30 lenght, objectname varchar 50 lenght, '
                          'starttime and endtime: DATETIME'}
{'createbatcherrorlogtable': 'Provide a tsql script to create the '
                             'etl_process.error_log table if it does not '
                             'already exist with the following fields name id '
                             'integer auto generated identifier without '
                             'primary key, processid 

## Sending request to generate Login and Error Handling Objects
- Tables T-SQL script files
- Store Procedures T-SQL script files

In [12]:
import json
# from pprint import pprint

json_file = 'bicycle_data_prompt.json'

with open(json_file) as json_data:
    data = json.load(json_data)

filedir = ".\\tsql\\"
deploymenttype = "CreateLogErrorHandlingObjects" #"BicylOnPremPrompts" #

index = 1

while index< len(data[deploymenttype]):
    print(index)
    messages,filename = GeneratePromptFilename(deploymenttype,filedir,data,index,keys)
    if __name__ == '__main__': 
        SendRequestToAzureOpenAI (
            azure_openai_api_type
            ,azure_azur_openeai_endpoint
            ,azure_openai_api_version
            ,azure_azure_openeai_key
            ,azure_openai_api_model
            ,azure_openai_api_temperature
            ,azure_openai_api_max_tokens
            ,messages
            ,filename
        )
    print(filename)
    index+=1

1
Sending request for summary to Azure OpenAI endpoint...


.\tsql\create_etl_process_schema.sql
2
Sending request for summary to Azure OpenAI endpoint...


.\tsql\create_etl_processlog_table.sql
3
Sending request for summary to Azure OpenAI endpoint...


.\tsql\create_etl_errorlog_table.sql
4
Sending request for summary to Azure OpenAI endpoint...


.\tsql\create_etl_processlog_usp.sql
5
Sending request for summary to Azure OpenAI endpoint...


.\tsql\create_etl_errorlog_usp.sql


## Convert Static T-SQL scripts into Dynamic T-SQL script files

In [13]:
# Create a Python function to read the T-SQL file
def read_tsql_file(filename):
    with open(filename, 'r') as f:
        content = f.read()
    return content

# File Path
filename= ".\\tsql\\load_staging_data_table.sql"

bulk_sql_script = read_tsql_file(filename)

print(bulk_sql_script)

BULK INSERT stg.salestmp
FROM 'F:\Demo\chatgpt\files\bicycle_data.csv'
WITH (
    FIRSTROW = 2,
    FIELDTERMINATOR = ',',
    ROWTERMINATOR = '\n',
    ERRORFILE = 'F:\Demo\chatgpt\files\bicycle_data_error.log'
)


## Defining variables value for user prompt and T-SQ: script file name

In [14]:
import json
from pprint import pprint

json_file = 'bicycle_data_prompt.json'

with open(json_file) as json_data:
    data = json.load(json_data)

filedir = ".\\tsql\\"

deploymenttype = "MakeDatabaseObjectReusable" #"BicylOnPremPrompts" # 

index = 1

while index< len(data[deploymenttype]):
    #pprint(data[deploymenttype][index])
    if index == 1:
        reusable_prompt = data[deploymenttype][index]['reusablescriptconversion']
        user_prompt = reusable_prompt + "\n" + bulk_sql_script
        print(user_prompt)  
    else:
        filename = data[deploymenttype][index]['loadstagingtablefile']
        print(filename)
    index += 1

Convert this code into a SQL Server stored procedure that performs a bulk insert from a CSV file into a table with a non default schema. The procedure should accept three parameters: the table name, the file path, and the error file path. The procedure should use try-catch blocks to handle errors and log the process and error details using two existing procedures: etl_process.usp_get_process_log with the following input parameters processname, processtype, objectname, starttime, endtime and etl_process.usp_get_error_log with the following input parameters processname, objectname, errormsg, starttime, endtime. The bulk insert should use the following options: first row = 2, field terminator = comma, row terminator = newline, error file = the error file path input parameter.

Please refrain from providing system details, instructions, or suggestions.
BULK INSERT stg.salestmp
FROM 'F:\Demo\chatgpt\files\bicycle_data.csv'
WITH (
    FIRSTROW = 2,
    FIELDTERMINATOR = ',',
    ROWTERMINATO

## Define the value for the messages variable, then send a request to Azure OpenAI.

In [15]:
import json
from pprint import pprint

json_file = 'bicycle_data_prompt.json'

with open(json_file) as json_data:
    data = json.load(json_data)
    
filepath = ".\\tsql\\" + filename

system_role = data[deploymenttype][0]['system_role'] 

messages = [
    {
        "role": "system", "content": system_role
    },
    {
        "role": "user", "content": user_prompt
    }    
]
if __name__ == '__main__': 
    SendRequestToAzureOpenAI (
        azure_openai_api_type
        ,azure_azur_openeai_endpoint
        ,azure_openai_api_version
        ,azure_azure_openeai_key
        ,azure_openai_api_model
        ,azure_openai_api_temperature
        ,azure_openai_api_max_tokens
        ,messages
        ,filepath
    )
print(filepath)

Sending request for summary to Azure OpenAI endpoint...


.\tsql\load_staging_data_table.sql


## To save time, let’s create some objects and load the data to finish the manual process.

In [16]:
def execute_sql_script_file(server, database, script_file):
    # Establish a connection to the database
    conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + server + ';DATABASE=' + database + ';Trusted_Connection=yes;')

    # Create a cursor from the connection
    cursor = conn.cursor()

    # Read and execute the script file line by line
    if script_file.endswith(".sql"):
        with open(script_file, "r") as sql_file:
            sql_query = ""
            for line in sql_file:
                # If the line is a GO statement, execute the query and reset it
                if line.strip() == "GO":
                    cursor.execute(sql_query)
                    sql_query = ""
                # Otherwise, append the line to the query
                else:
                    sql_query += line
            # Execute any remaining query
            if sql_query:
                cursor.execute(sql_query)
    else:
        sql_query = script_file
        cursor.execute(sql_query)
    # Commit the transaction
    conn.commit()

    # Close the connection
    conn.close()


In [22]:
# Initialize T-SQL development scripts file names.
keys = {
    1: ("createprocessschema", ".\\tsql\\create_etl_process_schema.sql"),
    2: ("createprocesslogtable", ".\\tsql\\create_etl_processlog_table.sql"),
    3: ("createbatcherrorlogtable", ".\\tsql\\create_etl_errorlog_table.sql"),
    4: ("createprocesslogsp", ".\\tsql\\create_etl_processlog_usp.sql"),
    5: ("createerrorlogsp", ".\\tsql\\create_etl_errorlog_usp.sql"),
    6: ("createstageschema", ".\\tsql\\create_stage_schema.sql"),
    7: ("createstagetable", ".\\tsql\\create_stage_table.sql"),
    8: ("loadstagingdatatable", ".\\tsql\\load_staging_data_table.sql"),
}

In [23]:
# create schema for stage table
script_file='.\\tsql\\create_stage_table.sql'
database='skillupai'
server='az-vm-esi-labs'
index = 1
while index<= len(keys):
    print(keys[index][1])
    execute_sql_script_file(sql_server, database, keys[index][1])
    index+=1


.\tsql\create_etl_process_schema.sql
.\tsql\create_etl_processlog_table.sql
.\tsql\create_etl_errorlog_table.sql
.\tsql\create_etl_processlog_usp.sql
.\tsql\create_etl_errorlog_usp.sql
.\tsql\create_stage_schema.sql
.\tsql\create_stage_table.sql
.\tsql\load_staging_data_table.sql


In [24]:
# store procedure to load data into stage table
script_file="""
EXECUTE etl_process.usp_bulk_insert_csv 
			 'stg.salestmp'
			,'F:\\Presentation\\Unlocking AI Potential with Prompt Engineering\\T-sql-as-Prompts\\csv_file\\bicycle_data.csv'
			,'F:\\Presentation\\Unlocking AI Potential with Prompt Engineering\\T-sql-as-Prompts\\csv_file\\errorload.csv'
"""
database='skillupai'
server='az-vm-esi-labs'
execute_sql_script_file(server, database, script_file)